<a href="https://colab.research.google.com/github/aiaaee/Analyse_Simple_Image/blob/main/Image_Analyzing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Amin")

Amin
